# Tâche 2 - Corriger les proverbes avec des modèles de langue N-grammes

L'objectif de cette tâche est de corriger des proverbes à l'aide de modèles de langue N-grammes en remplaçant un mot incorrect dans un proverbe connu. Il s'agit d'une tâche qui consiste à choisir le meilleur mot à insérer dans un texte en fonction du contexte de la phrase. 

Voir l'énoncé du travail #1 pour une description plus détaillée de cette tâche. 

Fichiers:
- *t2_proverbes.txt*: il contient plus de 3000 proverbes, un par ligne de texte. Vous utilisez ce fichier pour l'entraînement des modèles de langues N-grammes. 
- *t2_test1.json*: il contient des proverbes modifiés, les mots candidats de remplacement et la bonne formulation du proverbe. À utiliser pour évaluer la capacité des modèles de langue N-grammes à mettre les bons mots aux bons endroits. 

Consignes: 
- Utilisez NLTK pour construire les modèles de langue.
- Utilisez Spacy pour la tokenisation des textes et pour identifier les mots à remplacer. 
- N'oubliez pas de faire le rebourrage (*padding*) des proverbes avec des symboles de début et de fin.
- Faites un lissage de Laplace des modèles. 
- Ne pas modifier les fonctions *load_proverbs* et *load_tests*.
- Utilisez la variable *models* pour conserver les modèles après entraînement. 
- Ne pas modifier la signature de les fonctions *train_models* et *correct_proverb*.
- Des modifications aux signatures de fonctions entraîneront des pénalités dans la correction. 
- Vous pouvez ajouter des cellules au *notebook* et ajouter toutes les fonctions utilitaires que vous voulez. 

## Section 1 - Lecture des fichiers de données (proverbes et tests)

In [291]:
import json

# Ne pas modifier le chemin de ces 2 fichiers pour faciliter notre correction
proverbs_fn = "./data/t2_proverbes.txt"    
test_v1_fn = './data/t2_test1.json'

def load_proverbs(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        raw_lines = f.readlines()
    return [x.strip() for x in raw_lines]


def load_tests(filename):
    with open(filename, 'r', encoding='utf-8') as fp:
        test_data = json.load(fp)
    return test_data

In [292]:
proverbs = load_proverbs(proverbs_fn)

In [293]:
print("Nombre de proverbes pour l'entraînement: {}".format(len(proverbs)))
print("Un exemple de proverbe: " + proverbs[10])

Nombre de proverbes pour l'entraînement: 3108
Un exemple de proverbe: affaire menée sans bruit se fait avec plus de fruit


In [294]:
tests = load_tests(test_v1_fn)

In [295]:
import pandas as pd

def get_dataframe(test_proverbs):
    return pd.DataFrame.from_dict(test_proverbs, orient='columns', dtype=None, columns=None)

df = get_dataframe(tests)
df

,Masked,Word_list,Proverb
0,a beau mentir qui part de loin,"[vient, revient]",a beau mentir qui vient de loin
1,a beau dormir qui vient de loin,"[partir, mentir]",a beau mentir qui vient de loin
2,l’occasion forge le larron,"[fait, occasion]",l’occasion fait le larron
3,"endors-toi, le ciel t’aidera","[bouge, aide]","aide-toi, le ciel t’aidera"
4,"aide-toi, le ciel t’aura","[aidera, aide]","aide-toi, le ciel t’aidera"
5,"ce que femme dit, dieu le veut","[dit, veut]","ce que femme veut, dieu le veut"
6,"ce que femme veut, dieu le souhaite","[dit, veut]","ce que femme veut, dieu le veut"
7,bien mal acquis ne sait jamais,"[profite, fait]",bien mal acquis ne profite jamais
8,bon ouvrier ne déplace pas ses outils,"[fait, querelle]",bon ouvrier ne querelle pas ses outils
9,"pour le fou, c’était tous les jours fête","[est, es]","pour le fou, c’est tous les jours fête"


## Section 2 - Code pour repérer les mots qui pourraient être remplacés dans un proverbe modifié

Expliquez ici comment vous procédez pour identifier les mots d'un proverbe qui pourraient faire l'objet d'une substitution.  



Pour identifier les verbes dans les proverbes, nous utilisons `fr_dep_news_trf`. Le modèle identifie le type de mot à l'aide du part-of-speech tagging. Ensuite nous conservons seulement les mots étiquettés `VERB`.

In [296]:
!python -m spacy download fr_dep_news_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.8/397.8 MB 14.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_dep_news_trf')


In [297]:
# importer le model spacy de francais
import fr_dep_news_trf
analyzer_fr = fr_dep_news_trf.load()

to_replace = {
  "answers": [],
  "candidates": []
}

def find_candidates(tokens):
  canditates = []
  for i, token in enumerate(tokens):
    if token.tag_ == 'VERB':
      canditates.append((i, token.text))

  return canditates

def find_answer(masked_tokens, proverd_tokens):
  for i, (masked_token, proverb_token) in enumerate(zip(list(masked_tokens), list(proverd_tokens))):
    if masked_token.text != proverb_token.text:
      return (i, masked_token.text)
  

for index, row in df.iterrows():
  masked_tokens = analyzer_fr(row['Masked'])
  proverb_tokens = analyzer_fr(row['Proverb'])
  to_replace["answers"].append(find_answer(masked_tokens, proverb_tokens))
  to_replace["candidates"].append(find_candidates(masked_tokens))

print(to_replace["answers"])
print(to_replace["candidates"])
correct = 0
for answer, candidate in zip(to_replace["answers"], to_replace["candidates"]):
  if answer in candidate:
    correct += 1

print("Accuracy: ", correct/len(to_replace["answers"]))

/home/jonathan/Documents/ulaval/ift-7022-tp1/venv/lib/python3.12/site-packages/thinc/shims/pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dic

[(4, 'part'), (2, 'dormir'), (2, 'forge'), (0, 'endors'), (7, 'aura'), (3, 'dit'), (7, 'souhaite'), (4, 'sait'), (3, 'déplace'), (5, 'était'), (2, 'plaire'), (0, 'manger'), (4, 'courir'), (2, 'dormir'), (8, 'être'), (3, 'veut'), (3, 'faire'), (6, 'saisir'), (2, 'mord'), (7, 'navigue'), (2, 'mange'), (0, 'repose'), (10, 'retrouver'), (4, 'déplacent'), (11, 'déplacent'), (0, 'étudier')]
[[(1, 'beau'), (2, 'mentir'), (4, 'part')], [(0, 'a'), (1, 'beau'), (2, 'dormir'), (4, 'vient')], [(2, 'forge')], [(7, 'aidera')], [(0, 'aide'), (7, 'aura')], [(3, 'dit'), (7, 'veut')], [(3, 'veut'), (7, 'souhaite')], [(2, 'acquis'), (4, 'sait')], [(3, 'déplace')], [], [(0, 'dire'), (2, 'plaire')], [(0, 'manger'), (2, 'faire')], [(1, 'vaut'), (2, 'prévenir'), (4, 'courir')], [(1, 'vaut'), (2, 'dormir'), (4, 'guérir')], [(3, 'aide'), (7, 'peut')], [(3, 'veut'), (7, 'peut')], [(1, 'faut'), (3, 'faire'), (6, 'croire')], [(1, 'faut'), (3, 'voir'), (6, 'saisir')], [(2, 'mord'), (7, 'est')], [(2, 'vend'), (7, '

Nous obtenons une précision de 88% pour l'identification du verbe à remplacer. Nous avons aussi testé fr_core_news_lg, avec un résultat de 84% et fr_core_news_sm avec un résultat de 53%.

## Section 3 - Construction des modèles de langue N-grammes. 

La fonction ***train_models*** prend en entrée une liste de proverbes et construit les trois modèles unigramme, bigramme et trigramme.

Les 3 modèles entraînés sont conservés dans ***models***, un dictionnaire Python qui prend la forme 

<pre>
{
   1: modele_unigramme, 
   2: modele_bigramme, 
   3: modele_trigramme
}
</pre>

avec comme clé la valeur N du modèle et comme valeur le modèle construit par NLTK.

Expliquez ici comment vous procéder pour construire vos modèles avec NLTK, pour obtenir les n-grammes de mots, pour déterminer le vocabulaire, etc...

...

# Explication des procédés

Pour faire les modèles, nous débutons par extraire le vocabulaire du jeu de données, c'est-à-dire, tous les différents tokens utilisés. Ensuite, nous générons les unigrammes, bigrammes et trigrammes à partir du jeu de données. Lors de cette étapes, les listes de tokens pour chaque proverbe sont paddés avec les tokens <BOS> en début de phrase et <EOS> en fin de phrase. Une fois fait, nous initialisons les modèles avec un lissage de laplace et finalement, faire l'entrainement.

In [298]:
import nltk
from nltk.lm import Laplace
from nltk import pad_sequence, ngrams

BOS = '<BOS>'  # Jeton de début de proverbe
EOS = '<EOS>'  # Jeton de fin de proverbe

models = {} 

In [299]:
def build_vocabulary(proverbs):
    vocabulary = []
    for proverb in proverbs:
        tokens = [token.text for token in analyzer_fr(proverb)]
        vocabulary.extend(tokens)

    vocabulary = set(vocabulary)
    vocabulary.add(BOS)
    vocabulary.add(EOS)
    return vocabulary


def build_ngrams(proverbs, n=2):
    ngrams_list = []
    for proverb in proverbs:
        tokens = [token.text for token in analyzer_fr(proverb)]
        tokens = pad_sequence(tokens, n, pad_left=True, pad_right=True, left_pad_symbol=BOS, right_pad_symbol=EOS)
        ngrams_list.extend(list(ngrams(tokens, n)))
            
    return ngrams_list


In [300]:
def train_models(proverbs):
    """ Vous ajoutez à partir d'ici le code dont vous avez besoin
        pour construire les différents modèles N-grammes.
        Cette fonction doit construire tous les modèles en une seule passe.
        Voir les consignes de l'énoncé du travail pratique concernant les modèles à entraîner.

        Vous pouvez ajouter les fonctions/méthodes et variables que vous jugerez nécessaire.
        Merci de ne pas modifier la signature et le comportement de cette fonction (nom, arguments, sauvegarde des modèles).
    """

    # Votre code à partir d'ici...  
    #initialisation des structure pour stocker les modeles
    models={1:None,2:None,3:None}
    
    for n in range(1,4):
        print("Entrainement du modèle de n-gramme de taille: ",n)
        vocabulary = build_vocabulary(proverbs)
        ngrams_list = build_ngrams(proverbs, n)
        print(ngrams_list)

        model = Laplace(n)
        model.fit([ngrams_list], vocabulary_text=vocabulary)

        #sauvegarde du model 
        models[n]=model

    return models

 # Entraîner les modèles
models = train_models(proverbs)

Entrainement du modèle de n-gramme de taille:  1
[('a',), ('beau',), ('mentir',), ('qui',), ('vient',), ('de',), ('loin',), ('a',), ('beau',), ('se',), ('lever',), ('tard',), (',',), ('qui',), ('a',), ('bruit',), ('de',), ('se',), ('lever',), ('matin',), ('abandon',), ('fait',), ('larron',), ('abeilles',), ('sans',), ('reine',), (',',), ('ruche',), ('perdue',), ('abondance',), ('de',), ('biens',), ('ne',), ('nuit',), ('pas',), ('accord',), ('vaut',), ('mieux',), ('qu’',), ('argent',), ('accueille',), ('le',), ('pauvre',), ('avec',), ('bonté',), (',',), ('fût',), ('-il',), ('un',), ('infidèle',), ('achète',), ('en',), ('foire',), (',',), ('et',), ('vends',), ('à',), ('la',), ('maison',), ('acquiers',), ('bonne',), ('renommée',), (',',), ('et',), ('dors',), ('grasse',), ('matinée',), ('adieu',), ('paniers',), (',',), ('vendanges',), ('sont',), ('faites',), ('affaire',), ('menée',), ('sans',), ('bruit',), ('se',), ('fait',), ('avec',), ('plus',), ('de',), ('fruit',), ('affaires',), ('vois

## Section 4 - Corriger un proverbe

In [301]:
def correct_proverb(modified_proverb, word_list, n=3, criteria="perplexity"):
    """ Le paramètre criteria indique la mesure qu'on utilise 
        pour choisir le mot le plus approprié: "logprob" ou "perplexity".
        On retourne l'estimation de cette mesure sur le proverbe complet,
        c.-à-d. en utilisant tous les mots du proverbe.

        Le paramètre n désigne le modèle utilisé.
        1 - unigramme NLTK, 2 - bigramme NLTK, 3 - trigramme NLTK
        
        Cette fonction retourne la solution (le proverbe corrigé) et 
        la valeur de logprob ou perplexité (selon le paramètre en entrée de la fonction). 
    """

    # Votre code à partir d'ici. Vous pouvez modifier comme bon vous semble.
    
    # tokenisation du proverbe
    tokens = analyzer_fr(modified_proverb)
     
    candidates = find_candidates(tokens)
    if len(candidates) == 0:
        return None, None, None
    best_choice = None
    best_score = None
    best_i_candidate = None
    tokens_str = [token.text for token in tokens]

    tokens = list(pad_sequence(tokens_str, n, pad_left=True, pad_right=True, left_pad_symbol=BOS, right_pad_symbol=EOS))
    if criteria == "logprob":
        best_logprob = float('-inf')
        for i_candidate, _ in candidates:
            i_candidate = i_candidate + n - 1
            for word in word_list:
                tokens
                logprob = models[n].logscore(word, tokens[i_candidate-n+1:i_candidate])
                if logprob > best_logprob:
                    best_logprob = logprob
                    best_choice = word
                    best_i_candidate = i_candidate
            if best_choice is None:
                best_choice = word_list[0]
        best_score = best_logprob


    elif criteria == "perplexity":
        best_perplexity = float('inf')
        for i_candidate, _ in candidates:
            i_candidate = i_candidate + n - 1
            for word in word_list:
                ngram = list(ngrams(tokens[i_candidate-n+1:i_candidate] + [word], n))
                perplexity = models[n].perplexity(ngram)
                if perplexity < best_perplexity:
                    best_perplexity = perplexity
                    best_choice = word
                    best_i_candidate = i_candidate
            if best_choice is None:
                best_choice = word_list[0]
        best_score = best_perplexity


    if best_i_candidate == None:
        return None, None, None
    i = modified_proverb.find(tokens[best_i_candidate])
    # print("token: ", tokens_str[best_i_candidate])
    correted_proverb = best_choice.join([modified_proverb[:i], modified_proverb[i + len(tokens[best_i_candidate]):]])
    return best_choice, best_score, correted_proverb


# masked = "endors-toi, le ciel t’aidera"
masked =  "repose-toi plutôt sans souper, que de te lever avec des dettes"
word_list = ['lève', 'couche']
answer = correct_proverb(masked, word_list, n=3, criteria="perplexity")
print(answer, "\n")

masked =  "a beau dormir qui vient de loin"   
word_list = ['partir', 'mentir']    
correct_proverb(masked, word_list, n=3, criteria="perplexity")

('couche', 3758.9999999999986, 'couche-toi plutôt sans souper, que de te lever avec des dettes') 



('mentir', 2207.500000000001, 'a beau mentir qui vient de loin')

Un exemple pour illustrer l'utilisation de cette fonction

In [302]:
masked =  "bouge-toi, le ciel t’aidera"   
word_list = ['endors', 'aide']    
correct_proverb(masked, word_list, n=3, criteria="logprob")

('aide', -11.876133200043016, 'aide-toi, le ciel t’aidera')

In [303]:
masked =  "ce que femme veut, dieu le souhaite"
word_list = ["dit","veut"]    
correct_proverb(masked, word_list, n=3, criteria="logprob")

('veut', -11.106890045092218, 'ce que femme veut, dieu le souhaite')

In [304]:
masked =   "étudier peu, chasse beaucoup de maladies"   
word_list = [ "manger","parle"]    
correct_proverb(masked, word_list, n=3, criteria="perplexity")

('parle', 1252.9999999999995, 'parle peu, chasse beaucoup de maladies')

In [305]:
masked =  "le poisson mange par la tête"   
word_list = ["pourrit", "respire"]    
correct_proverb(masked, word_list, n=3, criteria="perplexity")

('pourrit', 2207.0000000000027, 'le poisson pourrit par la tête')

## Section 5 - Expérimentations et analyse de résultats

Menez votre expérimentation dans cette section. Décrivez les résultats obtenus et présentez l'évaluation obtenue sur le(s) fichier(s) de test. Vous pouvez ajouter le nombre de cellules que vous souhaitez. 

In [306]:
# La Fonction qui évalue les performances
def evaluate_models(tests, n, criteria):
    results = []

    for entree_test in tests:
        masked_proverb = entree_test['Masked']
        word_list = entree_test['Word_list']
        correct_solution = entree_test['Proverb']

        # Tokenisation du proverbe
        tokens = analyzer_fr(masked_proverb)

        # Vérifiez si find_candidate retourne des résultats valides
        candidate_info = find_candidates(tokens)
        if candidate_info is None:
            results.append({
                'masked_proverb': masked_proverb,
                'word_list': word_list,
                'correct_solution': correct_solution,
                'candidate_result': "Aucune solution trouvée",
                'score': None,
                'is_correct': False
            })
            continue
        
        # Compléter le proverbe
        candidate_result, score, corrected_proverb = correct_proverb(masked_proverb, word_list, n, criteria)

        # Vérification pour éviter de traiter un None
        if candidate_result is None:
            candidate_result = "Aucune solution trouvée"
            is_correct = False
        else:
            is_correct = (corrected_proverb == correct_solution)

        results.append({
            'masked_proverb': masked_proverb,
            'word_list': word_list,
            'correct_solution': correct_solution,
            'candidate_result': candidate_result,
            'corrected_proverb': corrected_proverb,
            'score': score,
            'is_correct': is_correct
        })

    return results


In [307]:
# Évaluons ici  les performances du modèle avec des ngrammes et des critères spécifiques
ngram_modeles = [1, 2, 3]
criteria = ["perplexity", "logprob"]

results_summary = []

for n in ngram_modeles:
    for crit in criteria:
        results = evaluate_models(tests, n, crit)
        accuracy = sum(result['is_correct'] for result in results) / len(results) * 100
        results_summary.append({
            'ngram_modeles': n,
            'criteria': crit,
            'accuracy': accuracy,
            'results': results
        })

# Affichons ici le résumé des  performances obtenu
for summary in results_summary:
    print(f"Modèle {summary['ngram_modeles']}-gramme, Critère {summary['criteria']}:")
    print(f"Précision: {summary['accuracy']:.2f}%")
    print(f"Exemple de proverbe corriger correctement:")
    correct_results = [result for result in summary['results'] if result['is_correct']]
    # ici nous avons afficher les trois premiers exemple qui sont  corrects
    for example in correct_results[:3]:  
        print(f"Proverbe masqué: {example['masked_proverb']}")
        print(f"Proverbe corriger: {example['corrected_proverb']}")
    print()

Modèle 1-gramme, Critère perplexity:
Précision: 15.38%
Exemple de proverbe corriger correctement:
Proverbe masqué: l’occasion forge le larron
Proverbe corriger: l’occasion fait le larron
Proverbe masqué: ce que femme dit, dieu le veut
Proverbe corriger: ce que femme veut, dieu le veut
Proverbe masqué: à qui dieu veut, nul ne peut nuire
Proverbe corriger: à qui dieu aide, nul ne peut nuire

Modèle 1-gramme, Critère logprob:
Précision: 15.38%
Exemple de proverbe corriger correctement:
Proverbe masqué: l’occasion forge le larron
Proverbe corriger: l’occasion fait le larron
Proverbe masqué: ce que femme dit, dieu le veut
Proverbe corriger: ce que femme veut, dieu le veut
Proverbe masqué: à qui dieu veut, nul ne peut nuire
Proverbe corriger: à qui dieu aide, nul ne peut nuire

Modèle 2-gramme, Critère perplexity:
Précision: 46.15%
Exemple de proverbe corriger correctement:
Proverbe masqué: a beau mentir qui part de loin
Proverbe corriger: a beau mentir qui vient de loin
Proverbe masqué: a b

# Explication des résultats de la section 5

La section 5 nous permet d'evaluer la performance de nos modéles en utilisant les fichiers du test fourni.chaque modéle a été testé en complétant des proverbes masqué.L'objectif est de calculer la précision, c'est a dire le pourcentage des verbes corrigés correctement . Pour chaque modéle nous avons appliqué deux critéres d'évaluation
- logprob: un critére qui est basé sur le logarithme de la probabilité du mot
- Perplexity: un critére qui est basée sur la mesure de la perplexité des modéles et qui évalue la capacité  du modéle a prédire le prochain mot
  
Nos résultats obtenu sont ensuite résumés et affichés, montrant la précision de chaque modèle selon le  critère utilisé, avec quelques exemples de proverbes complétés correctement.
Et ces  résultats nous montrent que :

Le modéle unigramm pour les critères logprob et de perplexity a obtenue une précision de 15.38%, ce qui indique que ce modele n'est pas très performant pour corriger correctement les proverbes masqués. Naturellement, comme le modèle n'utilise pas le contexte pour déterminer la probabilité du mot, il est incapable de bien identifier le mot à utiliser selon le proverbe et seulement la fréquence des mots dans le jeu de données complet pour déterminer le mot à prendre.

Les modèles bigramme et trigramme réussissent beaucoup mieux avec 46.15% et 61.54% respectivement. Le contexte a un impact significatif sur les performances des modèles. Si on prend en compte que l'identification des verbes n'obtient que 88%, cela signifie que le modèle trigrammes est capable de corriger 70% des cas si le verbe à changer est parmis la liste des mots candidats à changer.

Pour les trois modèles, les deux critères obtiennent les mêmes performances ce qui n'est pas surprennant puisque les mots qui ont une petite perplexité auront aussi une grande probabilitée et vice-versa.

## Section 6 - Partie réservée pour faire nos tests lors de la correction

Merci de ne pas modifier ni retirer cette section du notebook.  